In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from matplotlib_venn import venn3_unweighted
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (confusion_matrix, hinge_loss, log_loss,
                             make_scorer, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, learning_curve,
                                     train_test_split, validation_curve)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearnex.svm import SVC

<h1><center>Data Cleaning</center></h1>

In [ ]:
# Read in survey data.
df = pd.read_csv('survey_results_public.csv')

In [ ]:
# Filter out all non-students.
students = df[df['Employment'].str.contains('Student, full-time', na=False)]

In [ ]:
# Drop features with mostly missing values.
nalimit = len(students) * 0.4
students = students.dropna(axis='columns', thresh=nalimit)

# Drop survey-related features.
del students['ResponseId']
students = students.loc[:, ~students.columns.str.contains("SO|Survey")]

# Reset dataframe indices.
students = students.reset_index(drop=True)

<h1><center>Data Analysis</center></h1>

In [ ]:
# Explore the domain of each feature in the dataset.
def explore_domain(df: pd.DataFrame):
    for feature in df:
        responses = set()

        # Many survey questions have multi-answer responses. Split them
        # to count the number of unique responses to each question.
        for response in df[feature]:
            responses |= response_to_set(response)

        # Display the number of possible responses, how many missing
        # responses there are, and list each possible response.
        print(f'{feature} (Count: {len(responses)}, Null: '
              f'{df[feature].isna().sum()}): '
              f'{"; ".join(sorted(responses))}\n')


# Converts a multi-answer survey response into a set of sub-responses.
def response_to_set(response: str):
    return set(response.split(';')) if pd.notna(response) else set()


# Counts the number of sub-responses given to a multi-answer question.
def count_responses(response: str):
    return response.count(';') + 1 if pd.notna(response) else 0


# explore_domain(students)

<h1><center>Feature Selection</center></h1>

In [ ]:
# Select data that can either be counted or categorized into features.
countable_data = ['LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith']
categorical_data = ['MainBranch', 'EdLevel', 'Age']
encoding = [
    {
        'I used to be a developer by profession, but no longer am': 0,
        ('I am not primarily a developer, but I write '
         'code sometimes as part of my work'): 1,
        'I am learning to code': 2,
        'I code primarily as a hobby': 3,
        'I am a developer by profession': 4
    },
    {
        'Something else': 0,
        'Primary/elementary school': 1,
        ('Secondary school (e.g. American high school, '
         'German Realschule or Gymnasium, etc.)'): 2,
        'Some college/university study without earning a degree': 3,
        'Associate degree (A.A., A.S., etc.)': 4,
        'Professional degree (JD, MD, etc.)': 5,
        'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 6,
        'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 7,
        'Other doctoral degree (Ph.D., Ed.D., etc.)': 8
    },
    {
        'Prefer not to say': 0,
        'Under 18 years old': 1,
        '18-24 years old': 2,
        '25-34 years old': 3,
        '35-44 years old': 4,
        '45-54 years old': 5,
        '55-64 years old': 6,
        '65 years or older': 7
    }
]
os_data = ['OpSysPersonal use', 'OpSysProfessional use']

# Isolate the IDEs column; this will be used in the output vector.
output = 'NEWCollabToolsHaveWorkedWith'

# Generate features by counting the number of responses to a question.
data = students[countable_data].applymap(count_responses)
data.columns = data.columns.map(lambda x: x.removesuffix('HaveWorkedWith'))

# Generate features by encoding distinct survey responses as digits.
for i, feature in enumerate(categorical_data):
    data[feature] = students[feature].map(encoding[i])

# Generate boolean features based on employment status, whether the
# respondent uses WSL, and use of version control systems (i.e., Git).
data['Employed'] = students['Employment'].str.contains('Employed')

os = students[os_data].applymap(response_to_set)
os = os.apply(lambda x: set.union(*x), axis=1)
data['WSL'] = os.map({'Windows Subsystem for Linux (WSL)'}.issubset)

data['VCS'] = students['VersionControlSystem'].fillna("I don't use one")
data['VCS'] = data['VCS'].map(lambda x: x != "I don't use one")

# Increase feature based on whether the VCS was used in an editor/IDE.
data['VCS'] += students['VCInteraction'].str.contains('Code editor', na=0)

# Generate additional numeric features based on the number of operating
# systems used by the respondent and how many years they have coded.
data['OS'] = os.map(len)
data['YearsCode'] = students['YearsCode'].replace('Less than 1 year', 1)
data['YearsCode'] = data['YearsCode'].replace('More than 50 years', 50)

# Classify respondents by whether they use VS Code.
data['VSCode'] = students[output].str.contains('Visual Studio Code', na=0)

# Clean missing values and convert boolean features to integers.
data = data.fillna(0).astype(int)

# Generate the design matrix and output vector.
X = data.drop('VSCode', axis=1)
y = data['VSCode']

<h1><center>Feature Analysis</center></h1>

In [ ]:
# Plot a heatmap showing the pairwise correlation between features.
def plot_correlation_matrix(df: pd.DataFrame):
    corr = df.corr()
    plt.subplots(figsize=(10, 8))
    sns.heatmap(corr, annot=True)


# Select data for "want to"-based multi-answer questions.
other_countable_data = ['LanguageWantToWorkWith', 'DatabaseWantToWorkWith',
                        'PlatformWantToWorkWith', 'WebframeWantToWorkWith',
                        'MiscTechWantToWorkWith', 'ToolsTechWantToWorkWith']

# Count the number of responses to questions regarding what the
# respondent has worked with and what they want to work with.
df_count = students[countable_data + other_countable_data]
df_count = df_count.applymap(count_responses)

# There is a strong correlation between the number of responses
# to "have worked with" and "want to work with" questions.
# plot_correlation_matrix(df_count)

In [ ]:
# Plots a stacked bar graph showing the proportion of VS Code
# users to non-VS Code users in the given feature category.
def plot_vscode_by_category(df: pd.DataFrame, x: str):
    # Use confusion matrix to get VS Code users by category.
    cm = confusion_matrix(df[x], df['VSCode'])
    non_vscode_users = cm[:, 0]
    vscode_users = cm[:, 1]

    # Compute proportions of VS Code vs. non-VS Code.
    total = non_vscode_users + vscode_users
    prop_non_vscode = np.true_divide(non_vscode_users, total) * 100
    prop_vscode = np.true_divide(vscode_users, total) * 100

    # Get positions of bars on the x-axis.
    x_axis = sorted(df[x].unique())
    num_cat = len(x_axis)
    x_axis = x_axis if num_cat > 2 else ['No', 'Yes']
    r = range(num_cat)

    # Plot top and bottom bars.
    plt.figure(figsize=(5 * num_cat ** 0.25, 5 * num_cat ** 0.25))
    barWidth = 0.8
    plt.bar(r, prop_non_vscode, bottom=prop_vscode, color='#ff4000',
            edgecolor='white', width=barWidth, label="Doesn't Use VS Code")
    plt.bar(r, prop_vscode, color='#00bfff', edgecolor='white',
            width=barWidth, label="Uses VS Code")

    # Plot category ticks, axis labels, and legend.
    plt.xticks(r, x_axis)
    plt.xlabel(x, fontweight='bold')
    plt.ylabel("Survey Respondents (%)", fontweight='bold')
    plt.legend(loc='lower right')
    plt.show()


# for feature in X:
#     plot_vscode_by_category(data, feature)

In [ ]:
# Display the importance of each feature, in descending order.
def show_feature_importance(X: pd.DataFrame, y: pd.Series):
    # Run a tree-based classifier to determine feature importance.
    dt = DecisionTreeClassifier(random_state=42, criterion='log_loss')
    dt.fit(X, y)

    # Compose features/importances in a dataframe and display said data.
    fi_df = pd.DataFrame(zip(X.columns, dt.feature_importances_),
                         columns=['Feature', 'Importance'])
    fi_df.sort_values('Importance', ascending=False, inplace=True)
    display(fi_df.reset_index(drop=True))


# show_feature_importance(X, y)

In [ ]:
# Plot a pie chart describing the proportion of VS Code users.
def plot_vscode_users(y: pd.Series):
    colors = colors = ['#56a5d8', '#7b6ca7']
    plt.pie(y.value_counts(), colors=colors, autopct='%.2f%%', startangle=90)
    plt.title('Students Who Use Visual Studio Code')
    plt.legend(ncol=2, labels=['Uses VS Code', "Doesn't Use VS Code"],
               bbox_to_anchor=(1, 0), loc='best')
    plt.subplots_adjust(top=0.85)
    plt.show()


# The dataset is imbalanced. Most respondents use VS Code.
# plot_vscode_users(y)

<h2><center>Applying Machine Learning Algorithms</center></h2>

In [ ]:
# Split data into training (60%), validation (20%), and test (20%) sets.
def train_val_test_split(X, y, seed):
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.6,
                                                      random_state=seed)
    X_test, X_val, y_test, y_val = train_test_split(X_val, y_val,
                                                    test_size=0.5,
                                                    random_state=seed)

    return X_train, X_val, X_test, y_train, y_val, y_test


X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, 42)

# Data used to convert cross-validation to hold-out validation.
# Most sklearn functions with a cv parameter rely on a k-fold split.
# We replace this with a 2-fold split: the training/validation set.
cv = [(X_train.index, X_val.index)]

# Print the dimensions of each dataset.
# print(f'Training Data: {X_train.shape}')
# print(f'Testing Data: {X_test.shape}')
# print(f'Validation Data: {X_val.shape}')

In [ ]:
# Get the name of the final estimator in a pipeline.
def get_model_name(estimator):
    return list(estimator.named_steps)[-1]


# Get the name of a parameter used in a pipeline.
def get_param_name(parameter):
    return parameter.partition("__")[-1]


# Returns the set of labels this model misclassified during testing.
def misclassified_labels(estimator):
    return set(np.where(y_test != estimator.predict(X_test))[0])

In [ ]:
# Plot a validation curve by varying the given hyperparameter.
def plot_validation_curve(estimator, param_name, param_values, cost=None):
    # Compute the errors of the training and validation data
    # across various different hyperparameter values.
    train_err, val_err = validation_curve(estimator=estimator, X=X, y=y,
                                          param_name=param_name,
                                          param_range=param_values,
                                          cv=cv, scoring=cost)

    # Truncate parameter values to display them as ticks on the x-axis.
    x_axis = ['{0:.4f}'.format(p) for p in param_values]

    # Plot the validation curve.
    plt.plot(x_axis, train_err, color='#ba4562', label='Training Error')
    plt.plot(x_axis, val_err, color='#3e8674', label='Validation Error')
    plt.title('Validation Curve')
    plt.xticks(rotation=75)
    plt.xlabel('C')
    plt.ylabel('Cost')
    plt.grid()
    plt.legend()
    plt.show()

    # Return hyperparameter value that minimizes validation error.
    return param_values[val_err.argmin()]


# Plot a learning curve using the given estimator and cost function.
def plot_learning_curve(estimator, cost):
    # Compute the errors of the training and validation
    # data across several different training sizes.
    train_sizes = np.linspace(0.1, 1.0, 10)
    train_sizes, train_err, val_err = learning_curve(estimator=estimator,
                                                     X=X, y=y,
                                                     train_sizes=train_sizes,
                                                     cv=cv, scoring=cost)

    # Plot the learning curve.
    plt.plot(train_sizes, train_err, color='#b45f06', label='Training Error')
    plt.plot(train_sizes, val_err, color='#6aa84f', label='Validation Error')
    plt.title('Learning Curve')
    plt.xlabel('Training Examples')
    plt.ylabel('Cost')
    plt.grid()
    plt.legend()
    plt.show()


# Plot the ROC curve(s) using the given estimator(s).
def plot_roc_curve(*estimators):
    for estimator in estimators:
        # Compute the ROC and AUC using the prediction probabilities.
        y_proba = estimator.predict_proba(X_test)[:, 1]
        FPR, TPR, thresholds = roc_curve(y_test, y_proba)
        AUC = roc_auc_score(y_test, y_proba)
        model_name = get_model_name(estimator)

        # Plot the ROC curve. The AUC is listed in the legend.
        plt.plot(FPR, TPR, label=f'{model_name} (AUC = {AUC:.4})')

    # Plot the TPR=FPR line, title, axis labels, and legend.
    plt.plot([0, 1], [0, 1], color='darkred', linestyle='--')
    plt.title('Receiver Operating Characteristic')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.xlim([0.0, 1.001])
    plt.ylim([0.0, 1.001])
    plt.legend(loc='lower right')
    plt.show()


# Plot a grid search by varying the given parameters as specified.
def plot_grid_search(estimator, param_grid, cv=cv):
    # Perform the grid search.
    grid = GridSearchCV(estimator, param_grid, cv=cv).fit(X, y)
    scores = grid.cv_results_['mean_test_score']
    param_1, param_2 = tuple(param_grid)
    param_2_values = list(map(str, param_grid[param_2]))
    num_values = len(param_2_values)

    for idx, val in enumerate(param_grid[param_1]):
        # Get the grid search scores from varying the second
        # parameter and keeping the first parameter the same.
        idx *= num_values
        param_1_scores = scores[idx:idx + num_values]
        
        # Create a separate label for each value of the first parameter.
        label = f'{get_param_name(param_1)} = {val}'
        plt.plot(param_2_values, param_1_scores, marker='o', label=label)

    # Plot the second parameter on the x-axis.
    plt.title('Grid Search')
    plt.xlabel(get_param_name(param_2))
    plt.ylabel('Score')
    plt.legend()
    plt.show()

    # Return hyperparameter values that maximize the grid search score.
    return grid.best_params_

In [ ]:
# Prints the main classification metrics.
def print_classification_report(estimator):
    # Generate confusion matrix from predictions.
    y_pred = estimator.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = cm[0][0], cm[0][1], cm[1][0], cm[1][1]

    # Compute accuracy, precision, recall, and F1 score.
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * precision * recall / (precision + recall)

    # Print the unpacked confusion matrix and computed metrics.
    print(f'TP = {TP}, TN = {TN}, FP = {FP}, FN = {FN}')
    print(f'Accuracy = {accuracy}')
    print(f'Precision = {precision}')
    print(f'Recall = {recall}')
    print(f'F1 Score = {f1_score}')

<h1><center>Logistic Regression</center></h1>

In [ ]:
# Chain feature scaling and regularized logistic regression.
log_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('Logistic Regression', LogisticRegression())
])

# Log loss will be used as the cost function here.
log_cost = make_scorer(log_loss, needs_proba=True)

# The regularization strength, C ~ 1/λ, will be varied here.
reg = 'Logistic Regression__C'
reg_values = np.geomspace(0.001, 1, 25)

# Get best value of C from the validation curve.
C = plot_validation_curve(log_clf, reg, reg_values, log_cost)

In [ ]:
# Reapply logistic regression using the tuned hyperparameter.
log_clf.set_params(**{reg: C})

# Plot the learning curve.
plot_learning_curve(log_clf, log_cost)

In [ ]:
# Fit the model to the training data.
log_clf.fit(X_train, y_train)

# Plot the ROC curve and AUC score.
plot_roc_curve(log_clf)

In [ ]:
# Print classification metrics.
print_classification_report(log_clf)

<h1><center>Support Vector Machine</center></h1>

In [ ]:
# Chain feature scaling and support vector classification.
svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('SVM', SVC())
])

# Hinge loss will be used as the cost function here.
svm_cost = make_scorer(hinge_loss)

# Use a grid search to tune multiple hyperparameters.
param_grid = {
    'SVM__C': [0.1, 1, 10, 100, 1000],
    'SVM__gamma': [1, 0.1, 0.01, 0.001, 0.0001]
}
params = plot_grid_search(svm_clf, param_grid)

In [ ]:
# Test grid search results for reproducibility.
def plot_grid_searches(seed_range):
    # Insert initial grid search results.
    grid_results = pd.DataFrame(columns=list(map(get_param_name, params)))
    grid_results.loc[42] = params.values()

    # Perform a grid search for each seed in the given range.
    for i in seed_range:
        X_train, X_val, *_ = train_val_test_split(X, y, seed=i)
        seed_cv = [(X_train.index, X_val.index)]
        param_results = plot_grid_search(svm_clf, param_grid, seed_cv)
        grid_results.loc[i] = param_results.values()

    # Display parameter results in a table.
    display(grid_results)


# plot_grid_searches(range(5))

In [ ]:
# Reapply SVM using the tuned hyperparameters.
svm_clf.set_params(**params)

# Plot the learning curve.
plot_learning_curve(svm_clf, svm_cost)

In [ ]:
# Enable probability estimates (required for ROC curve).
svm_clf.set_params(**{'SVM__probability': True})

# Fit the model to the training data.
svm_clf.fit(X_train, y_train)

# Plot the ROC curve and AUC score.
plot_roc_curve(svm_clf)

In [ ]:
# Print classification metrics.
print_classification_report(svm_clf)

<h2><center>Logistic Regression with<br>Polynomial Features</center></h2>

In [ ]:
# Chain polynomial features, feature scaling, and logistic regression.
poly_clf = Pipeline([
    ('polyfeatures', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('Polynomial LR', LogisticRegression(max_iter=1000))
])

# The regularization strength, C ~ 1/λ, will be varied here.
reg = 'Polynomial LR__C'
reg_values = np.geomspace(0.001, 1, 25)

# Get the best value of C from the validation curve.
C = plot_validation_curve(poly_clf, reg, reg_values, log_cost)

In [ ]:
# Reapply logistic regression using the tuned hyperparameter.
poly_clf.set_params(**{reg: C})

# Plot the learning curve.
plot_learning_curve(poly_clf, log_cost)

In [ ]:
# Fit the model to the training data.
poly_clf.fit(X_train, y_train)

# Plot the ROC curve and AUC score.
plot_roc_curve(poly_clf)

In [ ]:
# Print classification metrics.
print_classification_report(poly_clf)

<h1><center>Analyze Success</center></h1>

In [ ]:
# Now, we can compare the success of each model used.
models = [log_clf, svm_clf, poly_clf]

# Plot the ROC curves and AUC scores of each model.
plot_roc_curve(*models)

In [ ]:
# Plot the errors each model makes in a three-way Venn diagram.
misclassified_sets = tuple(map(misclassified_labels, models))
model_names = tuple(map(get_model_name, models))
venn3_unweighted(misclassified_sets, model_names)
plt.show()

In [ ]:
# Sample false positives from the given set of error indices.
def show_false_positives(error_idx):
    df = data_test.loc[list(error_idx)].query('VSCode == 0')
    print(f'False Positives = {len(df)}')
    display(df.sample(5, random_state=42) if len(df) >= 5 else df)


# Sample false negatives from the given set of error indices.
def show_false_negatives(error_idx):
    df = data_test.loc[list(error_idx)].query('VSCode == 1')
    print(f'False Negatives = {len(df)}')
    display(df.sample(5, random_state=42) if len(df) >= 5 else df)


# Combine the design matrix and output vector for the test data.
data_test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)
shared_errors = set.intersection(*misclassified_sets)
all_errors = set.union(*misclassified_sets)
log_errors, svm_errors, poly_errors = misclassified_sets

# Display samples from the shared errors across learning methods.
print('Shared Errors:')
show_false_negatives(shared_errors)
show_false_positives(shared_errors)

# Display samples from the unique errors of each learning method.
for model_name, clf_errors in zip(model_names, misclassified_sets):
    print(f'{model_name} Errors:')
    unique_errors = all_errors.copy()

    for misclassified in misclassified_sets:
        if clf_errors is not misclassified:
            unique_errors -= misclassified

    show_false_negatives(unique_errors)
    show_false_positives(unique_errors)